In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import sys
np.set_printoptions(threshold=sys.maxsize)


# Preprocessing data

In [49]:
df=pd.read_csv('/Users/vedantgupta/Documents/summer project/data-analysis-with-technical-indicators-1/nifty_sensex_close_data/M&M.NS_2011_to_2023.csv')

In [101]:
#input data
np.random.seed(0)
df_train_inp= df[df['Date'] < '2013-01-01']
df_train_inp = df_train_inp.drop(['Date'], axis=1)
train_inp_arr= df_train_inp.to_numpy()
np.random.shuffle(train_inp_arr)
X_train = train_inp_arr 
X_train.shape

(491, 4)

In [100]:
#output data
np.random.seed(0)
df_train_out= df.iloc[491:534,4:]
train_out_arr= df_train_out.to_numpy()
np.random.shuffle(train_out_arr)
y_train = train_out_arr
y_train.shape

(43, 1)

In [87]:
#validataion input data
np.random.seed(0)
df_val_inp= df.iloc[491:739,1:]
val_inp_arr= df_val_inp.to_numpy()
np.random.shuffle(val_inp_arr)
X_val = val_inp_arr
X_val.shape

(248, 4)

In [91]:
#validation output data
np.random.seed(0)
df_val_out= df.iloc[739:762,4:]
val_out_arr= df_val_out.to_numpy()
np.random.shuffle(val_out_arr)
y_val = val_out_arr
y_val.shape

(23, 1)

In [110]:
df_close=df['M&M.NS_close']
df_close
df_close = df_close.astype(float)
df_close.shape

#training array
train_arr=np.arr([])


(3204,)

# model 